<a href="https://colab.research.google.com/github/hmm0403/Cuda/blob/main/vec_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-mfrb_x0v
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-mfrb_x0v
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=253437cef861bf85cf418391f66a77950577ae6f7cac195fa3ba84eefa815a14
  Stored in directory: /tmp/pip-ephem-wheel-cache-syvbo_cl/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin
#%reload_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [15]:
%%cu
#include <cstdio>
#include <cstdlib>

#define CHECK_CUDA(call)                                                 \
  do {                                                                   \
    cudaError_t status_ = call;                                          \
    if (status_ != cudaSuccess) {                                        \
      fprintf(stderr, "CUDA error (%s:%d): %s:%s\n", __FILE__, __LINE__, \
              cudaGetErrorName(status_), cudaGetErrorString(status_));   \
      exit(EXIT_FAILURE);                                                \
    }                                                                    \
  } while (0)

__global__ void vec_add_kernel(const int *A, const int *B, int *C, int N) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < N) C[i] = A[i] + B[i];
  //printf("%d\t",i);
}

int main() {
  int N = 16381;
  int *A = (int *) malloc(N * sizeof(int));
  int *B = (int *) malloc(N * sizeof(int));
  int *C = (int *) malloc(N * sizeof(int));
  int *C_ans = (int *) malloc(N * sizeof(int));

  for (int i = 0; i < N; i++) {
    A[i] = rand() % 1000;
    B[i] = rand() % 1000;
    C_ans[i] = A[i] + B[i];
  }

  // TODO: Run vector addition on GPU
  int *A_gpu, *B_gpu, *C_gpu;
  cudaMalloc(&A_gpu, N * sizeof(int));
  cudaMalloc(&B_gpu, N * sizeof(int));
  cudaMalloc(&C_gpu, N * sizeof(int));

  cudaMemcpy(A_gpu, A, N * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(B_gpu, B, N * sizeof(int), cudaMemcpyHostToDevice);

  dim3 gridDim((N + 1024 - 1) / 1024);
  dim3 blockDim(1024);

  vec_add_kernel<<<gridDim, blockDim>>>(A_gpu, B_gpu, C_gpu, N);
  CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaMemcpy(C, C_gpu, N*sizeof(int), cudaMemcpyDeviceToHost));
  // Save the result in C

  for (int i = 0; i < N; i++) {
    if (C[i] != C_ans[i]) {
      printf("Result differ at %d: %d vs %d\n", i, C[i], C_ans[i]);
    }
  }
  // for(int i = 0; i < N; i++){
  //     printf("%d + %d = %d \n",A[i], B[i],C[i]);
  // }
  printf("Validation done.\n");

  return 0;
}


9696	9697	9698	9699	9700	9701	9702	9703	9704	9705	9706	9707	9708	9709	9710	9711	9712	9713	9714	9715	9716	9717	9718	9719	9720	9721	9722	9723	9724	9725	9726	9727	1024	1025	1026	1027	1028	1029	1030	1031	1032	1033	1034	1035	1036	1037	1038	1039	1040	1041	1042	1043	1044	1045	1046	1047	1048	1049	1050	1051	1052	1053	1054	1055	10272	10273	10274	10275	10276	10277	10278	10279	10280	10281	10282	10283	10284	10285	10286	10287	10288	10289	10290	10291	10292	10293	10294	10295	10296	10297	10298	10299	10300	10301	10302	10303	12576	12577	12578	12579	12580	12581	12582	12583	12584	12585	12586	12587	12588	12589	12590	12591	12592	12593	12594	12595	12596	12597	12598	12599	12600	12601	12602	12603	12604	12605	12606	12607	7200	7201	7202	7203	7204	7205	7206	7207	7208	7209	7210	7211	7212	7213	7214	7215	7216	7217	7218	7219	7220	7221	7222	7223	7224	7225	7226	7227	7228	7229	7230	7231	8352	8353	8354	8355	8356	8357	8358	8359	8360	8361	8362	8363	8364	8365	8366	8367	8368	8369	8370	8371	8372	8373	8374	8375	8376	8377	8378	8